# Fire Detection and Classification

This Jupyter notebook demonstrates the process of building and training a K-Nearest Neighbors (KNN) classifier and a neural network for fire detection and classification using a custom dataset. The dataset consists of images of fires and non-fires.
dataset is avialable at :https://www.kaggle.com/datasets/phylake1337/fire-dataset/data
## Table of Contents
1. [Imports and Setup](#imports-and-setup)
2. [Data Loading and Preprocessing](#data-loading-and-preprocessing)
3. [K-Nearest Neighbors Classifier](#k-nearest-neighbors-classifier)
4. [Neural Network Model](#neural-network-model)

## Imports and Setup

We start by importing the necessary libraries and modules.

```python
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump
from tensorflow.keras import models, layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
```

## Data Loading and Preprocessing

The dataset is loaded and preprocessed using OpenCV. The images are resized to 32x32 pixels, normalized, and flattened to a 1D array. The feature vectors and labels are stored in separate lists.

```python
feature_vectors = []
all_labels = []

for i, address in enumerate(glob.glob("fire_dataset\\*\\*")):
    img = cv2.imread(address)
    img = cv2.resize(img, (32, 32))
    img = img / 255.0
    img = img.flatten()
    feature_vectors.append(img)
    label = address.split("\\")[1]
    all_labels.append(label)

    if i % 100 == 0:
        print(f"[INFO] {i} images processed")
```

## K-Nearest Neighbors Classifier

A K-Nearest Neighbors classifier is trained on the dataset and evaluated for accuracy. The trained classifier is then saved to a file.

```python
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, all_labels, train_size=0.8)

clf = KNeighborsClassifier()
clf.fit(X_train, y_train)

accuracy = clf.score(X_test, y_test)
print('Accuracy is: {:.2f}%'.format(accuracy * 100))

dump(clf, 'fire_knn_cls.joblib')
```

## Neural Network Model

The labels are encoded and converted to one-hot encoding for use in a neural network. A simple neural network model is created using TensorFlow's Sequential API. The model consists of an input layer, two hidden layers with ReLU activation functions, and an output layer with a softmax activation function.

```python
le = LabelEncoder()
encoded_labels = le.fit_transform(all_labels)
one_hot_labels = to_categorical(encoded_labels)

x_train, x_test, y_train, y_test = train_test_split(feature_vectors, one_hot_labels, test_size=0.2)

net = models.Sequential([
    layers.Dense(300, activation='relu', input_dim=3072),
    layers.Dense(40, activation='relu'),
    layers.Dense(2, activation='softmax')
])

print(net.summary())

net.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

H = net.fit(x_train, y_train, batch_size=16, validation_data=(x_test, y_test), epochs=1)
```

The model is compiled with the SGD optimizer and categorical cross-entropy loss function. The model is then trained for 1 epoch.

In [ ]:
import numpy as np
import cv2
import glob
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from joblib import dump
from tensorflow.keras import models, layers
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Initialize lists to hold feature vectors and labels
feature_vectors = []
all_labels = []

# Read and process images from the dataset
for i, address in enumerate(glob.glob("fire_dataset\\*\\*")):
    img = cv2.imread(address)  # Read the image
    img = cv2.resize(img, (32, 32))  # Resize the image to 32x32 pixels
    img = img / 255.0  # Normalize the image
    img = img.flatten()  # Flatten the image to a 1D array
    feature_vectors.append(img)  # Append the feature vector
    label = address.split("\\")[1]  # Extract the label from the file path
    all_labels.append(label)  # Append the label

    # Print progress every 100 images
    if i % 100 == 0:
        print(f"[INFO] {i} images processed")

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(feature_vectors, all_labels, train_size=0.8)

# Train a K-Nearest Neighbors classifier
clf = KNeighborsClassifier()
clf.fit(X_train, y_train)

# Evaluate the classifier
accuracy = clf.score(X_test, y_test)
print('Accuracy is: {:.2f}%'.format(accuracy * 100))

# Save the trained classifier
dump(clf, 'fire_knn_cls.joblib')

# Encode the labels
le = LabelEncoder()
encoded_labels = le.fit_transform(all_labels)

# Convert the labels to one-hot encoding
one_hot_labels = to_categorical(encoded_labels)

# Split the data into training and testing sets for the neural network
x_train, x_test, y_train, y_test = train_test_split(feature_vectors, one_hot_labels, test_size=0.2)

# Create a neural network model
net = models.Sequential([
    layers.Dense(300, activation='relu', input_dim=3072),
    layers.Dense(40, activation='relu'),
    layers.Dense(2, activation='softmax')
])

# Print the model summary
print(net.summary())

# Compile the neural network model
net.compile(optimizer='SGD', loss='categorical_crossentropy', metrics=['accuracy'])

# Train the neural network model
H = net.fit(x_train, y_train, batch_size=16, validation_data=(x_test, y_test), epochs=1)
